In [ ]:
channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4']

def resample_epoch(epoch_data, original_sfreq, target_sfreq):
    """
    Resample a single epoch of EEG data from the original sampling frequency to a target sampling frequency.

    Args:
        epoch_data (ndarray): Array containing the EEG data for a single epoch.
        original_sfreq (float): Original sampling frequency of the epoch data.
        target_sfreq (float): Target sampling frequency for resampling.

    Returns:
        ndarray: Resampled EEG data.
    """
    original_num_samples = epoch_data.shape[0]
    original_time = np.arange(original_num_samples) / original_sfreq

    target_num_samples = int(original_num_samples * target_sfreq / original_sfreq)
    target_time = np.linspace(0, original_time[-1], target_num_samples)

    resampled_data = signal.resample(epoch_data, target_num_samples, axis=0)

    return resampled_data

In [ ]:
import numpy as np
from scipy import signal
from mne.preprocessing import ICA

class Preprocess:
    def __init__(self, sfreq, ica_components=8, low_freq_cutoff=1, high_freq_cutoff=30, notch_freq=60, target_sfreq=100, channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4']):
        self.sfreq = sfreq
        self.ica_components = ica_components
        self.low_freq_cutoff = low_freq_cutoff
        self.high_freq_cutoff = high_freq_cutoff
        self.notch_freq = notch_freq
        self.target_sfreq = target_sfreq
        self.channels = channels
        self.ica = None
        
    def resample_epoch(self, epoch_data, available_channels=['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4']):
        original_sfreq = self.sfreq
        target_sfreq = self.target_sfreq
        original_num_samples, num_channels = epoch_data.shape

        # Check if all the specified channels are present in the available channels
        missing_channels = set(self.channels) - set(available_channels)
        if missing_channels:
            raise ValueError("The following channels are not available in the data: {}".format(", ".join(missing_channels)))

        channel_indices = [available_channels.index(channel) for channel in self.channels]
        selected_epoch_data = epoch_data[:, channel_indices]

        target_num_samples = int(original_num_samples * target_sfreq / original_sfreq)
        target_time = np.linspace(0, original_time[-1], target_num_samples)

        resampled_data = signal.resample(selected_epoch_data, target_num_samples, axis=0)

        return resampled_data


    def remove_low_frequencies(self, eeg_epoch):
        nyquist_freq = 0.5 * self.sfreq
        high_freq_cutoff = self.low_freq_cutoff / nyquist_freq
        b, a = signal.butter(4, high_freq_cutoff, btype='highpass')
        filtered_epoch = signal.filtfilt(b, a, eeg_epoch, axis=1)
        return filtered_epoch

    def remove_high_frequencies(self, eeg_epoch):
        nyquist_freq = 0.5 * self.sfreq
        low_freq_cutoff = self.high_freq_cutoff / nyquist_freq
        b, a = signal.butter(4, low_freq_cutoff, btype='lowpass')
        filtered_data = signal.filtfilt(b, a, eeg_epoch, axis=1)
        return filtered_data

    def remove_notch_noise(self, eeg_epoch):
        nyquist_freq = 0.5 * self.sfreq
        notch_bandwidth = 1.0  # Width of the notch filter band (adjust as needed)
        notch_freq_norm = self.notch_freq / nyquist_freq
        b, a = signal.iirnotch(notch_freq_norm, notch_bandwidth)
        filtered_data = signal.filtfilt(b, a, eeg_epoch, axis=1)
        return filtered_data

    def apply_ica(self, eeg_epoch):
        if self.ica is None:
            self.ica = ICA(n_components=self.ica_components, random_state=0)
            self.ica.fit(eeg_epoch)
        cleaned_data = self.ica.apply(eeg_epoch)
        return cleaned_data

    def apply_average_reference_epoch(self, epoch_data):
        average_reference = np.mean(epoch_data, axis=0)
        referenced_epoch_data = epoch_data - average_reference
        return referenced_epoch_data

    

In [3]:
available_channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3','P4', ]
channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4']
missing_channels = set(channels) - set(available_channels)
if missing_channels:
    raise ValueError("The following channels are not available in the data: {}".format(", ".join(missing_channels)))

ValueError: The following channels are not available in the data: P4